In [3]:
# import necessary libraries
from requests import get
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from time import time 
from IPython.core.display import clear_output
from warnings import warn
import datetime

## Small test to automate re-scraping if it fails

In [13]:
# flag variable to check the scrape
# if unsuccessful scrape, try again
unsuccessful = True
#count = 0
while unsuccessful:
    url = 'https://www.yelp.com/biz/tian-tian-hainanese-chicken-rice-singapore-7?start=0'
    # make request to the site
    response = get(url)
    html_soup = BeautifulSoup(response.text,'html.parser')
    
    # find all with the same class tag
    res_containers = html_soup.find_all('div', class_='review__373c0__13kpL border-color--default__373c0__3-ifU')
    print(type(res_containers))
    # check the number entries in a page and to determine the next page value for skipping
    print(len(res_containers))
    
    # pause the loop
    sleep(randint(2,5))
    
    if len(res_containers) != 0:
        unsuccessful = False

<class 'bs4.element.ResultSet'>
20


In [14]:
# testing
# values in the tags of the second entry
first_comment = res_containers[1]
#first_comment

In [15]:
# get name of first customer
cust_name = first_comment.div.find('a', class_='link__373c0__1G70M link-color--inherit__373c0__3dzpk link-size--inherit__373c0__1VFlE').text
cust_name

'Vivian D.'

## Get restaurant type at the header

In [97]:
res_containers1 = html_soup.find_all('span', class_='display--inline__373c0__3JqBP margin-r1__373c0__zyKmV border-color--default__373c0__3-ifU')
#print(res_containers1[2]) # the corresponding res words are there
res_type = res_containers1[2].find_all('a', class_= 'link__373c0__1G70M link-color--inherit__373c0__3dzpk link-size--inherit__373c0__1VFlE')
print("Restaurant type:", end=' ')
res_list = []
for res in res_type:
    print(res.get_text(), end=', ')
    res_list.append(res.get_text())

Restaurant type: Hainan, Chicken Shop, 

In [10]:
# get restaurant name
res_name = html_soup.find('h1', class_='heading--h1__373c0__dvYgw undefined heading--inline__373c0__10ozy').text
print(f"Restaurant name: {res_name}")

Restaurant name: Tian Tian Hainanese Chicken Rice


## Actual Scraping

In [81]:
# get the start value for every new page in yelp
start = [str(i) for i in range(0,241,20)]
# debug
print(start, end=' ')

['0', '20', '40', '60', '80', '100', '120', '140', '160', '180', '200', '220', '240'] 

In [82]:
# redeclaring lists to store data in
res_names = []
res_types = []

# multiple values
cust_names_iter = [] # a conditioning list to check if scrape is unsuccessful

cust_names = []
cust_ratings = []
cust_dates = []
cust_comments = []

# number of requests
REQUESTS = 2

# flag variable to check the scrape
# if unsuccessful scrape, try again
unsuccessful = True

# preparing the monitoring of the loop
start_time = time()
requests = 1 # start request with request 1

# for every comment in the interval of 20
for pageStart in start:
    
    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if requests > REQUESTS:
        warn('Number of requests was greater than expected.')
        break
        
    unsuccessful = True
    fail_count = 0
    while unsuccessful:
        # make a get request
        response = get(f'https://www.yelp.com/biz/jumbo-seafood-singapore-4?start={pageStart}')
        
        # pause the loop
        sleep(randint(3,5))

        # monitor the requests
        elapsed_time = time() - start_time
        print(f'Request {requests}; Frequency: {requests/elapsed_time} requests/s')
        clear_output(wait = True)

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # get the comment container for all 20 comments in a page
        comment_containers = page_html.find_all('div', class_='review__373c0__13kpL border-color--default__373c0__3-ifU')
        
        if len(comment_containers) != 0:
            print(f"Request {requests}: SUCCESS --> Failed Count: {fail_count}")
            unsuccessful = False
        else:
            fail_count+=1
            #print(f"Request {requests}: unsuccessful scrape") # debug
            pass
    requests += 1
    
    # for every comments in 20
    for com in comment_containers:
        
        # scrape the customer name
        cust_name = com.div.find('a', class_='link__373c0__1G70M link-color--inherit__373c0__3dzpk link-size--inherit__373c0__1VFlE').text
        cust_names.append(cust_name)
        
        # scrape the customer ratings
        cust_rating = com.find('div', class_='arrange__373c0__2C9bH gutter-1__373c0__2l5bx vertical-align-middle__373c0__1SDTo border-color--default__373c0__3-ifU').span.div['aria-label']
        cust_ratings.append(cust_rating)
        
        # scrape customer date
        cust_date_raw = com.find('div', class_='arrange__373c0__2C9bH gutter-1__373c0__2l5bx vertical-align-middle__373c0__1SDTo border-color--default__373c0__3-ifU').text
        #cust_date = datetime.datetime.strptime(cust_date_raw, '%m/%d/%Y').strftime('%Y-%m-%d')
        cust_dates.append(cust_date_raw)
        
        # scrape the customer comment
        cust_comment_raw = com.find_all('div', class_='margin-b2__373c0__abANL border-color--default__373c0__3-ifU')
        if len(cust_comment_raw) != 1:
            temp = cust_comment_raw[1].text
        else:
            temp = cust_comment_raw[0].text
            
        cust_comment = temp.replace(u'\xa0', u'')
        cust_comments.append(cust_comment)
                

Request 1: SUCCESS --> Failed Count: 3
Request 2: SUCCESS --> Failed Count: 2


C:\Users\nicholasneo78\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: UserWarning: Number of requests was greater than expected.


In [83]:
# check the number of entries
len(cust_names)

40

In [84]:
# debug: take a look at the data entries
#print(cust_names)
#print(cust_ratings)
#print(cust_dates)
#print(cust_comments)

## Fit values scraped into a dataframe

In [104]:
cust_review = pd.DataFrame({
    'Reviewer\'s Name': cust_names,
    'Rating': cust_ratings,
    'Date': cust_dates,
    'Comment': cust_comments,
})
cust_review['Rating'] = cust_review['Rating'].map(lambda x: x.rstrip('star rating'))
cust_review['Date'] = cust_review['Date'].map(lambda x: x.rstrip('Updated review'))
# convert date time to standardised format
cust_review['Date'] = pd.to_datetime(cust_review['Date'], format='%m/%d/%Y')

# append 2 new columns to identify the store name and its type
cust_review['Restaurant Type'] = str(res_list)
cust_review['Restaurant Name'] = res_name

print(cust_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Reviewer's Name  40 non-null     object        
 1   Rating           40 non-null     object        
 2   Date             40 non-null     datetime64[ns]
 3   Comment          40 non-null     object        
 4   Restaurant Type  40 non-null     object        
 5   Restaurant Name  40 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 2.0+ KB
None


In [105]:
cust_review.head(20)

,Reviewer's Name,Rating,Date,Comment,Restaurant Type,Restaurant Name
0,Margot C.,5,2020-03-19,My boyfriend and I came here for dinner while ...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice
1,Kira N.,4,2020-02-10,196 SGD (Singapore Dollar) for some crab and f...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice
2,Diane C.,2,2019-11-19,Holy cow! Totally wish we stuck with the stree...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice
3,Steve H.,5,2019-12-08,Jumbo seafood is the place to go for crabs. My...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice
4,Lan T.,4,2019-08-21,During my 3 days stay in Singapore in March of...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice
5,Kevin C.,4,2019-08-19,A Singapore institution that never disappoints...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice
6,Prima S.,4,2020-03-08,I was recommended this restaurant by someone w...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice
7,Yi Z.,5,2019-05-03,I am aware that Jumbo Seafood is a total touri...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice
8,Rita K.,2,2019-04-05,Underwhelmed by the flavors and quality of the...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice
9,Lyssa A.,5,2019-03-16,Came for the chili crab and was NOT disappoint...,"['Hainan', 'Chicken Shop']",Tian Tian Hainanese Chicken Rice


In [106]:
cust_review.to_csv('./data/chicken_rice_comments.csv', index=False)